![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2: Basic Chisel

#### Written by Stevo Bailey, Edward Wang, and Richard Lin
[stevo@berkeley.edu](mailto:stevo@berkeley.edu), [edwardw@berkeley.edu](mailto:edwardw@berkeley.edu), [richard.lin@berkeley.edu](mailto:edwardw@berkeley.edu)

## Introduction

Now that you're familiar with Scala, it's time to learn Chisel. Chisel is a hardware construction language (HCL), similar to a hardware design language but embedded in a higher level, functional programming language. The details and benefits of an HCL will be covered in later modules. This module introduces basic Chisel, which is quite similar to Verilog. Once the basics are understood, you can use them to construct parameterized generators and take full advantage of the language.

Throughout this module, the goal will be to produce a linear feedback shift register (LFSR) in hardware. If you are unfamiliar with LFSRs, skim the [Wikipedia page](https://en.wikipedia.org/wiki/Linear-feedback_shift_register) first. To do this, we will need to understand how combinational logic, sequential logic, operators, and data types are all handled in Chisel.

At a high level, the basic 4-bit LFSR circuit we will be building will look like:

![LFSR-4](images/lfsr4.svg)

## Setup

Download and import Chisel by running this cell:

In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

// A wrapper function that provides a easy way to get the generated Verilog code for a Chisel module
// Don't worry about the implementation of this.
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

# Hello, World: A Simple Example
This section will present the simple hardware module, a simple test, and how to run it. It will contain many things that you will not understand, and that is ok. We want you to take away the broad strokes, so you can continually return to this complete working example to reinforce what you've learned.

## A Simple Module

Like Verilog, we can declare module definitions in Chisel. The following example is a Chisel `Module`, `Simple` that has one input, `in`, and one output, `out`, and inside it combinationally connects `in` and `out`, so `in` drives `out`.

What is neat about HCL's is we can use the underlying programming language as a scripting language. For example, after declaring our Chisel module, we then use Scala to call the Chisel compiler to translate Chisel `Simple` into Verilog `Simple`. This process is called ***elaboration***.

In [ ]:
// Chisel Code: Declare a new module definition
class Simple extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

// Scala Code: Elaborate our Chisel design by translating it to Verilog
// Don't worry about understanding this code, it is very complicated Scala
println(getVerilog(new Simple))

Note that the Name of our module is `cmd<#>WrapperHelperSimple`, which is an artifact of running this tutorial in Jupyter. In your normal code, it should have the name Simple. This is an important lesson though - although Chisel does its best to preserve the names of your modules and other hardware components, sometimes it fails to do so.

## A Simple Tester

No hardware module should be complete without a tester. The following example is a Chisel test harness `SimpleTester` that passes values to an instance of `Simple`'s input port `in`, and checks that the same value is seen on the output port `out`.

Like before, we also use Scala to script our execution of the test.

In [ ]:
// Chisel Code: Declare a new tester for Simple modules
class SimpleTester(c: Simple) extends PeekPokeTester(c) {
  poke(c.io.in, 0)     // Set our input to value 0
  expect(c.io.out, 0)  // Assert that the output correctly has 0
  poke(c.io.in, 1)     // Set our input to value 1
  expect(c.io.out, 1)  // Assert that the output correctly has 1
  poke(c.io.in, 2)     // Set our input to value 2
  expect(c.io.out, 2)  // Assert that the output correctly has 2
}

// Scala Code: Calling Driver to instantiate Simple, SimpleTester, and execute the test
// Don't worry about understanding this code, it is very complicated Scala
val works = Driver(() => new Simple) {
  c => new SimpleTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


## Understanding Chisel vs Scala

The above examples compile Chisel into Verilog. It is important to understand what this process is, so that you as a designer can more clearly differentiate between what is hardware (Chisel) and what is scripting/parameterizations (Scala).

Elaboration is the process of constructing hardware when running a Scala/Chisel program. This hardware could be represented as a schematic, or a Verilog file with neither parameters nor generate statements. In essence, elaboration removes all scripting/parameterizing around a circuit, leaving only the bare description of hardware.

Let's look at some examples which clarify ***scripting*** values from ***hardware*** values.

In this first example, we have two Modules `ScalaPlus` and `VerilogPlus`. In both examples, we call the `+` operator. However, the first `+` is resolved during elaboration (`1 + 2 == 3`), while the second instantiates a Verilog `+` operator and explicitly trims the carry-out. *Note that the Verilog now contains many temporary nodes of the form* `_T_<#>`*.*

In [ ]:
// Chisel Code: Declare a new module definition
class ScalaPlus extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  io.out := (1 + 2).U  // `+` resolved during elaboration
}
println(getVerilog(new ScalaPlus))

class VerilogPlus extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  io.out := 1.U + 2.U  // `+` remains in Verilog
}
println(getVerilog(new VerilogPlus))

***What is going on here???*** When we elaborate a Chisel design, we are executing the Scala code that implements Chisel. It is designed in such a way that when it executes, it builds up an un-parameterized hardware circuit.

In `ScalaPlus`:
- <span style="color:red; font-family:monospace">(1 + 2)</span> is an evaluated during elaboration, resulting in `3`
- `(1 + 2)`<span style="color:red; font-family:monospace">.U</span> converts `3` into a 2-bit hardware constant, represented by Verilog's `2'h3`
- `out `<span style="color:red; font-family:monospace"> := </span>` (1 + 2).U` combinationally connects `3.U` to `out`, resulting in the Verilog `assign out = 2'h3;`.

In `ChiselPlus`:
- `1`<span style="color:red; font-family:monospace">.U</span> creates a 1-bit hardware constant, represented by Verilog's `1'h1`
- `2`<span style="color:red; font-family:monospace">.U</span> creates a 2-bit hardware constant, represented by Verilog's `2'h2`
- `1.U ` <span style="color:red; font-family:monospace">+</span> ` 2.U` creates a non-expanding hardware adder, represented by Verilog's `+` operator with logic to omit the carryout
- `out `<span style="color:red; font-family:monospace"> := </span> `1.U + 2.U` creates a combinational connection between the result of `+` with `out`, resulting in the Verilog `assign out = ...` statement

## A Quick Debugging Primer

There 3 major classes of bugs you may encounter:

- **Compile-time errors**, where the Scala compiler is unable to build your code. A syntax error is an example. This typically produces a stack trace which can help narrow down the potential causes. Searching the Internet for the error message is often productive, as there is a community around Scala.
- **Elaboration-time errors**, where Chisel operations have been invoked improperly. The code compiles and runs, but errors out before producing Verilog. Depending on the compilation stage of where this error occurs, some debugging information (like file and line numbers) may be available
- **RTL bugs**, where the design is legal but wrong. Chisel is unable to detect these, though some restrictions attempt to reduce the chances of this by disallowing dangerous or confusing constructs.

## Combinational Logic
### Data Types

Chisel adds a set of hardware types to the Scala type system. Chisel types eventually map to wires and logic. Example declarations of Chisel literals and variables in various types are given below. `B` stands for boolean, which is functionally the same as a one-bit `UInt`. `U` stands for `UInt` or unsigned integer. `S` stands for `SInt` or signed integer. Note that the width of an `SInt` includes the sign bit, so a 4-bit `SInt` ranges from -8 to 7. Notice how all Chisel variables are Scala `val`s. **Never use a Scala `var` for a hardware construct**, since the construct may never change once defined; only its value may change. Wires may be used for parameterized types.
```scala
// literals or constants:
val foo  = 4.U(8.W)        // unsigned literal with value 4 (8-bits wide)
val foob = "b00000100".U   // same as above, unsigned literal with value 4 (8-bits wide)
val bar  = -4.S(11.W)      // signed literal with value -4 (11-bits wide)
val qux  = true.B          // boolean literal with value true
    
// hardware types:
val foo2 = UInt(8.W)       // unsigned 8-bit wire (no value yet!)
val foo3 = Wire(UInt(8.W)) // same as above, unsigned 8-bit wire
val bar2 = SInt(11.W)      // signed 11-bit wire
val qux2 = Bool()          // boolean wire

// using wires with parameterized type:
class test[T<:Data](myType: T) extends Module {
  val io = IO(new Bundle {})
  val myWire = Wire(myType) // wire of type myType, a subset of Data
}
```

### Chisel Operators

Like Verilog, Chisel is written with modules, which contain IO and logic. The details of modules are discussed later. For now, just know that all custom modules extend the Module class, and the IO should be formatted as below. Inputs are wrapped in Input() functions, and outputs in Ouptut() functions. Inputs and outputs must be Chisel types. Below is an exmaple module which passes the inputs through to the outputs. Ports in the IO bundle are accessed by prepending them with `io`, as seen below.

In [ ]:
class ModuleExample extends Module {
  val io = IO(new Bundle {
    val myInput = Input(Bool())
    val myOutput = Output(Bool())
  })
  io.myOutput := io.myInput
}

An overview of Chisel operators is given below. For a more complete list, consult the [Chisel3 cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf), specifically the third column on the first page.

In [ ]:
class OperatorsExample extends Module {
  val io = IO(new Bundle {
    val a = Input(Bool())
    val b = Input(Bool())
    val z = Output(Bool())
  })

  // Colon equals assigns the value of the expression on the right-hand side to the variable on the left-hand side
  val foo = Wire(UInt(16.W))        // define a wire called foo of type unsigned integer with width 16 bits
  foo := io.a ^ io.b          // foo is assigned a XOR b (bitwise)
  io.z := foo                 // assigns foo to the output z

  // Comparisons return a Chisel Bool type, NOT a Scala Boolean type!!
  val bar = io.a === io.b     // bar is 1 if a equals b, otherwise 0
  val qux = io.a =/= io.b     // qux is 1 if a does not equal b, otherwise 0
  val baz = io.a  >  io.b     // baz is 1 if a is greater than b, otherwise 0 (see also <, >=, and <=)

  // Bit-level and ternary operators 
  val corge = io.a(10, 4)     // extracts bits 4, 5, 6, 7, 8, and 9 from the 16-bit input a (0 indexed)
  val waldo = io.a(15)        // extracts bit 15 from the 16-bit input a (the MSB)
  val plugh = Cat(io.a, io.b) // concatenates the two 16-bit inputs a and b into plugh, a 32-bit UInt
  val xyzzy = Mux(io.a < io.b, io.a, io.b) // assigns to xyzzy input a if a is less than b, otherwise input b

  // Math operators
  val quux = io.a + io.b      // assigns to quux the sum of a and b; the width of quux is inferred
  val fred = io.a - io.b      // assigns to fred the difference between a and b; the width of fred is inferred
  val thud = io.a * io.b      // assigns to thud the product of a and b
}

Now that you have a cursory overview of operators, let's practice using them and data types with the following exercise. Write code that passes the test. First scroll down and view the tester, then modify the OperatorsExercise module in the indicated locations. Remember our goal of producing an LFSR. The input is a 4-bit word that we'll later store in a shift register. The result is the output of the LFSR (which will be fed back into the input of the shift register). Your task is to make the logic for an LFSR of the polynomial x<sup>4</sup>+x<sup>3</sup>+1. Note that the 1 is implicit, so bit 0 of the input is really x<sup>1</sup>.

Graphically, the circuit looks like:

![LFSR-4 combinational](images/lfsr4combinational.svg)

In [ ]:
// Create the module
class OperatorsExercise extends Module {
  val io = IO(new Bundle {
    val input = Input(UInt(4.W))
    val result = Output(Bool())
  })

  // YOUR CODE HERE
  // Remember: input and result are inside io, so access them with io.input and io.result
  // Also, there's no need to typecast between 1-bit UInts and Bools
  // Hint: you should use the assignment operator :=
}

// Create the test
class OperatorsExerciseTester(c: OperatorsExercise) extends PeekPokeTester(c) {
  for (i <- 0 until math.pow(2,4).toInt) {
    poke(c.io.input, i)
    expect(c.io.result, (i >> 2 & 1) ^ (i >> 3 & 1))
  }
}

// Run the test!
val works = Driver(() => new OperatorsExercise) {
  c => new OperatorsExerciseTester(c)
}
println(works)

## Sequential Logic

_In the previous section, we built the combinational part of the LFSR - that is, given the simultaneous output of all the shift register bits, generate the bit to feed back in. In this section, we will build the sequential part, the shift registers themselves._

The basic stateful element in Chisel is the register, or `Reg`. It keeps its current value through a clock cycle, and can optionally be updated with a new value that becomes visible at the next clock edge.

_Chisel connects an implicit clock line to all state elements, and an implicit reset line to state elements with an _ initial _ value. There are constructs for overriding this, for example in multi-clock designs, but we won't go into those here._

A register can be constructed given a data type. For instance, a 2-bit unsigned integer register without initialization:
```scala
val myReg = Reg(UInt(2.W))
```

or with initialization (to 1 decimal). Both versions below are valid and do the same thing:
```scala
val myReg = RegInit(UInt(2.W), 1.U)
val myReg = RegInit(1.U(2.W))
```

Its value can be updated with the `:=` operator. For example, this increments the register's value:
```scala
myReg := myReg + 1.U
```

Note that when read, it gives its current (stored) value, but the assignment doesn't take effect until the next clock edge.

Below is an example which uses a register to implement a 2-bit wide counter, initializing at 1 and overflowing from 3 to zero. Run the block and take a look at the output trace.

![LFSR-4](images/counter2.svg)

In [ ]:
// Module containing the register
class MyCounter extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg := myReg + 1.U
  io.out := myReg

  printf("out=%d\n", io.out)
  
  printf(p"${io}\n")
}

// Testvector
class MyCounterTester(c: MyCounter) extends PeekPokeTester(c) {
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 3)
  step(1)
  expect(c.io.out, 0)
  step(1)
  expect(c.io.out, 1)
  step(1)
}

// Driver run invocation
Driver(() => new MyCounter) {
  c => new MyCounterTester(c)
}

One important note is that Chisel distinguishes between types (like `UInt`) and hardware nodes (like the literal `2.U`, or the output of `myReg`). While
```scala
val myReg = Reg(UInt(2.W))
```
is legal because a Reg needs a data type as a model,
```scala
val myReg = Reg(2.U)
```
is an error because `2.U` is already a hardware node and can't be used as a model.

Similarly, the `RegInit` examples have a type as their first argument and hardware node as the second argument. The hardware node is required to give a concrete initialization value.

Given that, now build a module that implements a shift register for your LFSR. Specifically:
- Each element is a single bit wide.
- Has 4 stages.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the most significant bit being the last element of the shift register and the least significant bit being the first element of the shift register. `Cat` may come in handy.
- **The output initializes at `b0001`.**
- Shifts each clock cycle (no enable signal).

![4-stage shifter](images/shifter4.svg)

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [ ]:
class MyShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), 1.U)  // 4-bit register as unsigned integer, initialized to 1 (b0001)

  // YOUR CODE HERE
  // You may find the bit extract (apply), bit shift (<<), and bit concatenation (Cat) operations useful.
  // Don't forget to assign the module output!

  printf("%x\n", io.out)
}

In [ ]:
class MyShiftRegisterTester(c: MyShiftRegister) extends PeekPokeTester(c) {
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

val works = Driver(() => new MyShiftRegister) {
  c => new MyShiftRegisterTester(c)
}

Chisel also offers several more variants of the register constructor, for example with a `next` field or with an `enable` signal, but these are outside the scope of this introductory tutorial.

## Conditional Logic and Wires
_We've now seen the basics of building combinational and sequential logic circuits, but those are very low-level primitives. Chisel provides a higher level of abstraction for some operations._

### Conditional Register Updates
Chisel operators can be gated by putting them inside a `when` conditional block. In our counter example above, if we also added a `count` input signal, we could gate the incrementing of the counter as follows:

```scala
when (io.count) {
  myReg := myReg + 1.U
}
```

In the case the `when` condition isn't met, the code inside simply isn't "run" on that cycle. Registers hold their state by default, so `myReg` will "pause" when the `when` condition isn't met, unless it is assigned later on to a different value.

When also has an `.elsewhen` and `.otherwise` construct, similar to `else if` and `else`. For example, if we wanted to count the number of cycles `count` is consecutively high, we could have it reset in the `.otherwise` condition:

```scala
when (io.count) {
  myReg := myReg + 1.U
} .otherwise  {
  myReg := 0.U
}
```

_Note that there is a `.` before the `otherwise`, this due to limitations of Scala syntax._

### Wires
While we've seen the update operator `:=` used combinationally on Module boundary `IO`s, it can be also applied to Module-internal `Wire`s. Here's an example of the counter above, styled slightly differently to make the combinational logic explicit and using `Wire`s.
```scala
val myReg = RegInit(UInt(2.W), 1.U)
val nextReg = Wire(UInt(2.W))  // constructed similarly to Regs
nextReg := myReg + 1.U
myReg := nextReg
```

### Conditional Combinational Updates
Conditional blocks can also be used to generate combinational logic. Let's say now that we want a counter that counts 0, 1, 2, then resets. In this case, since we need to reset on 2, we can no longer depend on the implicit integer overflow and need to make the logic explicit:
```scala
val nextReg = Wire(UInt(2.W))
when (myReg === 2.U) {
  nextReg := 0.U
} .otherwise {
  nextReg := myReg + 1.U
}
```

When there are multiple update operations that can apply at the same time, the last one takes priority. So the above example can be equivalently rewritten by unconditionally updating `nextReg` with the counter increment, then overriding it in the overflow case:

```scala
val nextReg = Wire(UInt(2.W))
nextReg := myReg + 1.U
when (myReg === 2.U) {
  nextReg := 0.U
}
```

There is also `WireInit`, a shorthand which declares and initializes a `Wire`:

```scala
val nextReg = WireInit(UInt(2.W), myReg + 1.U)
when (myReg === 2.U) {
  nextReg := 0.U
}
```

Whether to use the first, second, or third forms is mainly a matter of style and readability. They are all functionally equivalent to the structral form with a mux:

```scala
val nextReg = Wire(UInt(2.W))
nextReg := Mux(myReg === 2.U, 0.U, myReg + 1.U)
```

### A Gated, 3-tick Counter
Putting it all together, here's the full example:

In [ ]:
// Module containing the register
class MyNewCounter extends Module {
  val io = IO(new Bundle {
    val count = Input(Bool())
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)

  val nextReg = Wire(UInt(2.W))
  when (myReg === 2.U) {
    nextReg := 0.U
  } .otherwise {
    nextReg := myReg + 1.U
  }
  
  when (io.count) {
    myReg := nextReg
  }

  io.out := myReg
    
  printf("out=%d\n", io.out)
}

// Testvector
class MyNewCounterTester(c: MyNewCounter) extends PeekPokeTester(c) {
  poke(c.io.count, 1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 0)
  step(1)
  expect(c.io.out, 1)
  poke(c.io.count, 0)  // test pause
  step(1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 1)
  poke(c.io.count, 1)  // unpause
  step(1)
  expect(c.io.out, 2)
  step(1)
}

// Driver run invocation
Driver(() => new MyNewCounter) {
  c => new MyNewCounterTester(c)
}

The `when` blocks are high-level constructs that ease the generation of muxes. The actual circuit would look something like:

![Counter 3](images/counter3.svg)

Take your shift register design from above, but modify it so that it only shifts when the update signal is asserted.

As with before, a basic testbench and module skeleton are provided for you.

In [ ]:
class MyGatedShiftRegister extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val update = Input(Bool())
    val out = Output(UInt(4.W))
  })
  val s1 = RegInit(Bool(), true.B)

  // YOUR CODE HERE
  // don't forget to update the first register!

  printf("%x\n", io.out)  
}

In [ ]:
class MyGatedShiftRegisterTester(c: MyGatedShiftRegister) extends PeekPokeTester(c) {
  poke(c.io.update, 1)
  expect(c.io.out, 1)  // b0001
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 2)  // b0010
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 5)  // b0101
  poke(c.io.update, 0)  // pause
  step(1)
  expect(c.io.out, 5)
  step(1)
  expect(c.io.out, 5)
  step(1)
  poke(c.io.update, 1)  // unpause
  poke(c.io.in, 1)
  step(1)
  expect(c.io.out, 11)  // b1011
  poke(c.io.in, 0)
  step(1)
  expect(c.io.out, 6)  // b0110
}

Driver(() => new MyGatedShiftRegister) {
  c => new MyGatedShiftRegisterTester(c)
}

## Modules
_Without re-use, any non-trivial RTL design wouldn't get very far. This section goes into the basics of re-use with Chisel._

In all the example code, you've probably noticed that all your classes extend the `Module` class. Chisel `Module`s are very much like Verilog `module`s, they are instantiable blocks of logic that contain well-defined IO ports.

As all `Module`s are classes, they can be instantiated using the `new` object syntax. However, they also must be wrapped in a `Module(...)` call. For example, to instantiate the 4-state counter example way above:
```scala
val myModule = Module(new MyCounter())
```

You can then reference its IO objects in its parent class. For example, if you wanted to know when `myModule`'s count was 3, you could:
```scala
val isCount3 = myModule.io.out === 3.U
```

### A 4-cycle tick
This example below uses the `MyCounter` class from above to build a block that generates a one high cycle for every 4 clock cycles by comparing its output.

![4-ticker](images/myTicker.svg)

In [ ]:
// Module containing the register
class MyTicker extends Module {
  val io = IO(new Bundle {
    val out = Output(Bool())
  })
  
  val myModule = Module(new MyCounter())
  io.out := myModule.io.out === 3.U
}

// Testvector
class MyTickerTester(c: MyTicker) extends PeekPokeTester(c) {
  expect(c.io.out, 0)  // counter at 1
  step(1)
  expect(c.io.out, 0)  // counter at 2
  step(1)
  expect(c.io.out, 1)  // counter at 3
  step(1)
  expect(c.io.out, 0)  // counter at 0
  step(1)
  expect(c.io.out, 0)  // counter at 1
  step(1)
  expect(c.io.out, 0)  // counter at 2
  step(1)
  expect(c.io.out, 1)  // counter at 3
  step(1)
}

// Driver run invocation
Driver(() => new MyTicker) {
  c => new MyTickerTester(c)
}

Modules are just one form of composition and re-use in Chisel; you'll learn more soon.

## Now You Try: The Full LFSR

Now, build a full LFSR by taking your `MyGatedShiftRegister` and `OperatorsExercise` modules and instantiating them in a parent module. As usual, the module skeleton and a testbench is provided for you.

![LFSR-4](images/lfsr4blocks.svg)

In [ ]:
class MyLfsr extends Module {
  val io = IO(new Bundle {
    val update = Input(Bool())
    val out = Output(UInt(4.W))
  })
  
  val myShifter = Module(new MyGatedShiftRegister())
  myShifter.io.update := io.update
  
  // YOUR CODE HERE
  // Don't forget to make the other needed connections to myShifter!
  
  printf("%x\n", io.out)
}


In [ ]:
// Testvector
class MyLfsrTester(c: MyLfsr) extends PeekPokeTester(c) {
  poke(c.io.update, 1)
  expect(c.io.out, 1)
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 4)
  step(1)
  expect(c.io.out, 9)
  step(1)
  expect(c.io.out, 3)
  step(1)
  expect(c.io.out, 6)
  step(1)
  expect(c.io.out, 13)
  step(1)
  expect(c.io.out, 10)
  step(1)
  expect(c.io.out, 5)
  step(1)
  expect(c.io.out, 11)
  poke(c.io.update, 0)  // pause
  step(1)
  expect(c.io.out, 11)
  step(1)
  expect(c.io.out, 11)
  poke(c.io.update, 1)  // unpause
  step(1)
  expect(c.io.out, 7)
  step(1)
  expect(c.io.out, 15)
  step(1)
  expect(c.io.out, 14)
  step(1)
  expect(c.io.out, 12)
  step(1)
  expect(c.io.out, 8)
  step(1)
  expect(c.io.out, 1)  // LFSR restarts
  step(1)
  expect(c.io.out, 2)
  step(1)
  expect(c.io.out, 4)
  step(1)
  expect(c.io.out, 9)
  step(1)
}

// Driver run invocation
Driver(() => new MyLfsr) {
  c => new MyLfsrTester(c)
}

## A Few More Constructs

### Vec (Hardware Arrays)

The Chisel `Vec` is a **type** representing a fixed-length array of some other type. What this means is that we can use a  `Vec` anywhere we would use other types like `UInt` or `Bool`.

Example:
```scala
val vectorRegister = Reg(Vec(4, UInt(8.W))
// This is analogous to the following scalar version.
val scalarRegister = Reg(UInt(32.W))
```

We can do almost all of the usual things that we can do with arrays.

```scala
// Read from Vec
val firstElement = scalarRegister(0)
// Write to Vec
scalarRegister(3) := nextValueForLastElement
// Or even do this using dynamic/runtime indexing!
val myElement = scalarRegister(io.myAddr)
```

We can even flatten/deflatten a `Vec` into a flat `UInt`.

```scala
// Flatten a Vec into a UInt
val flatVec = vectorRegister.asUInt
// Expand a UInt into a Vec (of the appropriate size)
val vecScalar = scalarRegister.asTypeOf(Vec(4, UInt(8.W))
```

### Memories

Memories are given special treatment in Chisel since hardware implementations of memory vary greatly. For example, FPGA memories are instantiated quite differently from ASIC memories. Chisel defines a memory abstraction that can map to either simple Verilog behavioural descriptions or to instances of memory modules that are available from external memory generators provided by foundry or IP vendors.

Chisel supports random-access memories via the `Mem` construct. Writes to `Mem`s are **combinational/asynchronous-read, sequential/synchronous-write**. These `Mem`s will likely be synthesized to register banks, since most SRAMs in modern technologies (FPGA, ASIC) tend to no longer support combinational (asynchronous) reads.

Chisel also has a construct called `SyncReadMem` for **sequential/synchronous-read, sequential/synchronous-write** memories. These `SyncReadMem`s will likely be synthesized to technology SRAMs (as opposed to register banks).

#### Creating Memories

Memories are declared with two parameters - the number of entries (addresses) and the datatype of each entry.

```scala
val mem = SyncReadMem(
  1024, /* This memory will have 1024 entries (10-bit address). */
  UInt(32.W) /* Each entry is of width 32. (data width = 32) */
)
```

Ports into Mems are created by applying a `UInt` index.  A 1024-entry register file with one write port and one sequential/synchronous read port might be expressed as follows:

```scala
val width:Int = 32
val addr = Wire(UInt(width.W))
val dataIn = Wire(UInt(width.W))
val dataOut = Wire(UInt(width.W))
val enable = Wire(Bool())

// assign data...

// Create a synchronous-read, synchronous-write memory (like in FPGAs).
val mem = SyncReadMem(1024, UInt(width.W))
// Create one write port and one read port.
mem.write(addr, dataIn)
dataOut := mem.read(addr, enable)
```

A combinational/asynchronous read memory (likely synthesized to a register bank) looks like the following snippet. Note how similar it is to the synchronous case.

```scala
// Create a (likely) register-bank backed combinational-read memory.
val mem = Mem(1024, UInt(32.W))
// Create one write port and one read port.
mem.write(addr, dataIn)
dataOut := mem.read(addr, enable)
```

For the rest of this section on memories, we will concern ourselves with sequential/synchronous memories.

#### Single-Ported Memories

Chisel can also infer other features such as single ports and masks directly with the Mem/SyncReadMem construct.

Single-ported SRAMs can be inferred when the read and write conditions are mutually exclusive in the same `when` chain:

``` scala
val mem = SyncReadMem(2048, UInt(32.W))
when (write) { mem.write(addr, dataIn) }
.otherwise { dataOut := mem.read(addr, read) }
```

If the same `Mem` address is both written and sequentially read on the same clock edge, or if a sequential read enable is cleared, then the read data is undefined.

#### Masks

Chisel memories also support write masks for subword writes. Chisel will infer masks if the data type of the memory is a vector. To infer a mask, specify the `mask` argument of the `write` function which creates write ports. A given masked length is written if the corresponding mask bit is set. For example, in the example below, if the 0th bit of mask is true, it will write the lower 8 bits of the corresponding address.

```scala
val dataOut = Wire(Vec(4, UInt(8.W)))
val dataIn = Wire(Vec(4, UInt(8.W)))
val mask = Wire(Vec(4, Bool()))
val enable = Wire(Bool())
val readAddr = Wire(UInt(10.W))
val writeAddr = Wire(UInt(10.W))

// ... assign values ...

// Create a 32-bit wide memory that is byte-masked.
val mem = SyncReadMem(1024, Vec(4, UInt(8.W)))
// Create one masked write port and one read port.
mem.write(writeAddr, dataIn, mask)
dataOut := mem.read(readAddr, enable)
```

#### Exercises

TODO: write exercises

e.g. create a masked single port memory. Test that we can write to only one byte using masks

e.g. create a 4-ported memory with 2 read ports and 2 write ports. Test reading and writing.

e.g. implement a shift register using mem?

### BlackBoxes

Chisel can interface with the Verilog world through `BlackBox`es. A **`BlackBox`** is a Chisel-declaration of an external Verilog module. The `BlackBox` interface extends Chisel to interoperate with non-native primitives or existing Verilog IP. For example:

```scala
// A latch. Chisel doesn't have a native latch primitive, but a BlackBox allows us to
// use them in Chisel designs.
class Latch extends BlackBox {
  val io = IO(new Bundle {
    val d = Input(Bool())
    val q = Output(Bool())
    val en = Input(Bool()) // transparent high
  })
}

// A register with an asynchronous reset. Though Chisel doesn't natively support this,
// BlackBoxes allow us to use this Chisel designs.
class NegEdgeFlipFlop extends BlackBox {
  val io = IO(new Bundle {
    val d = Input(Bool())
    val q = Output(Bool())

    val clock = Input(Clock())
    val reset = Input(Bool())
  })
}

// A register with an asynchronous reset. Same deal with BlackBox.
class AsyncResetReg extends BlackBox {
  val io = IO(new Bundle {
    val d = Input(Bool())
    val q = Output(Bool())
    val en = Input(Bool())

    val clock = Input(Clock())
    val reset = Input(Bool())
  })
}

// BlackBoxes don't have to involve clocks or state elements.
// They could even refer to hard macros or even more efficient (circuit-level)
// implementations.
// See later for how to implement proper abstraction for them.
class EladAlonHighSpeedAdder extends BlackBox {
  val io = IO(new Bundle {
    val a = Input(UInt(1024.W))
    val b = Input(UInt(1024.W))
    val out = Output(UInt(1024.W))
  })
}
```

TODO: provide Verilog implementations

TODO: create Chisel designs as exercises (implement logic to instantiate blackboxes)

TODO: run blackboxes in verilator tester

### Bundles

Let's revisit the simple adder example (listed conveniently below for you):

In [ ]:
class RevisitingAdders extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(32.W))
    val b = Input(UInt(32.W))
    val out = Output(UInt(32.W))
  })
  io.out := io.a + io.b
}

Notice that the input and output ports of the module are wrapped in `IO` and `new Bundle`.

```scala
val io = IO(new Bundle {
  val a = Input(UInt(32.W))
  val b = Input(UInt(32.W))
  val out = Output(UInt(32.W))
})
```

Don't worry too much about the `IO()` portion - that simply tells Chisel that the following `Bundle` (which we will talk about) is the list of I/O ports for the module, just like how `Input()` tells Chisel that a given port is an Input.

The `new Bundle` portion is interesting though, since the IO of a module is a `Bundle`. Chisel's `Bundle` construct allows us to easily group together related pieces of data to form data structures, like `struct` in C or SystemVerilog.

Chisel Bundles can actually be created as full classes in their own right (as opposed to the anonymous class we saw earlier with `new Bundle`).

```scala
class Coordinate extends Bundle {
  val x = SInt(32.W)
  val y = SInt(32.W)
}

class FloatingPoint extends Bundle {
  val sign = UInt(1.W)
  val exponent = UInt(8.W)
  val mantissa = UInt(23.W)
}

class CacheLine extends Bundle {
  val valid = Bool()
  val tag = UInt(27.W)
  val data = Vec(1 << 4, UInt(8.W))
}

class Complex extends Bundle {
  // Look, we can even use Bundles in other Bundles.
  // [cue inception music]
  val real = new FloatingPoint()
  val imag = new FloatingPoint()
}
```

We can use them in all the usual ways.

```scala
val a = Wire(new Coordinate)
a.x := 0.S
a.y := 0.S
val b = Wire(new Coordinate)
b.x := 5.S
b.y := 5.S
val out = Wire(new Coordinate)
// Logic to add a and b, and store the output in out.
// YOUR CODE HERE

val float1 = Wire(new FloatingPoint)
val float2 = Wire(new FloatingPoint)
val out = FloatingPointAdder(float1, float2) // magic

val myCache = Reg(Vec(32, new CacheLine)) // For educational reasons only
when (writeCache) {
  myCache(index).valid := true.B
  myCache.tag := tag
  myCache.data := data
}
```

Chisel's `Bundle` construct also support elements with directionality. For example:

In [ ]:
// For actual code, you should use https://chisel.eecs.berkeley.edu/api/index.html#chisel3.util.ReadyValidIO
// This is for educational purposes only.
// Written from the producer's perspective (data is an output).
class ReadyValidUInt32 extends Bundle {
  val ready = Input(Bool())
  val valid = Output(Bool())
  val data = Output(UInt(32.W))
}

class ReadyValidUInt32Pipe extends Bundle {
  val in = (new ReadyValidUInt32).flip
  val out = new ReadyValidUInt32
}

We can use these like other bundles, except that we don't have to wrap them in `Input()` or `Output()`.

In [ ]:
// An adder that is slow. It should take the given number of cycles to compute
// and not be ready to accept any further input.
class SlowAdder extends Module {
  val io = IO(new Bundle {
    // Number of cycles to delay.
    val delay = Input(UInt(8.W))
    val in = (new ReadyValidUInt32).flip
    val out = new ReadyValidUInt32
  })

  // YOUR CODE HERE
}

// A fast adder. It is always ready to add.
class FastAdder extends Module {
  val io = IO(new ReadyValidUInt32Pipe) // We can even use a predefined bundle as the IO!

  // YOUR CODE HERE
}

In [ ]:
// TODO: write tests

## Scala vs. Chisel

When writing Chisel code, sometimes it can be confusing as to what belongs to Scala and what belongs to Chisel, and what generates hardware and what does not.

At the simplest level, Scala is a general-purpose programming language, and Chisel is a software library/DSL written in Scala. However, as hardware engineers, it can sometimes be confusing to recognize the difference between the two. Here is an overview of the Chisel process:

1. Write Chisel (write a series of instructions on how to construct hardware).
2. Execute Chisel code (executes the series of instructions and generate a concrete piece of hardware).
3. 'Run' elaborated hardware (runs a simulation/synthesis of the concrete hardware).

In order to better understand the difference and clarify the confusion, we have to talk about the difference between **compile-time/elaboration-time** and **run-time/hardware**.

* **Compile-time/elaboration-time** things "live" in stage 2 above. They are variables/constants/parameters that are used in the 'receipe' for constructing the hardware, but don't actually exist in the final hardware in any way.
* **Run-time/hardware** things live in stage 3 above. They actually exist in some form in the physical hardware.

Here is a quick table summarizing the similarly-named constructs in Scala/Chisel:

| Scala (Compile-time) | Chisel (Hardware/Run-time) |
|----------------------|----------------------------|
| Int/BigInt           | UInt, SInt                 |
| Boolean              | Bool                       |
| Seq/Array/Vector     | Vec                        |

## Chisel Motivation as Verilog++

*Adapted from the [Chisel DAC 2012 paper](http://chisel.eecs.berkeley.edu/chisel-dac2012.pdf)*

Many of the problems associated with Verilog/VHDL arise from the fact that they were originally developed as hardware simulation languages, and later re-purposed for hardware synthesis. Chisel was designed to make writing hardware better by providing a new foundation that eliminates many of these legacy issues.

* Synthesizable designs must be inferred from a subset of the language, complicating  tool  development  and  designer  education.
 - It is very easy to write non-synthesizable designs in Verilog inadvertently. For example: accidentally inferring latches by forgetting to specify a case. In constrast, a Chisel `Reg()` is always a register and cannot accidentally become a latch.
 - This makes writing hardware more tedious that it needs to be - e.g. the lack of a register primitive and having to write the full `always @ (posedge clock) begin ... end`. In constrast, a Chisel register is a simple `Reg()`, and updating it is a matter of `reg := ...`.
* These languages also lack powerful abstraction and type facilities of modern software languages, making re-use hard.
 - For example, the lack of object-oriented programming to share and re-use code.
 - Or a strict type system to catch errors early.
 - Or implict wire declarations which can allow typos to sneak through.
 - Inability to have proper libraries of re-usable components.
 - Inability to define powerful, extensive interfaces for re-use.
 - In constrast, Chisel's Scala foundation is a modern object-oriented programming language with the right features to make abstraction and re-use easy.
* It is hard to write powerfully parametrized circuits in only Verilog, necessitating all sorts of Perl/Python scripts that generate Verilog through snippets.
 - Chisel's Scala foundation makes it easy to write flexible and powerful generators, like the [Rocket Chip Generator](https://github.com/freechipsproject/rocket-chip), or [BOOM](https://github.com/ucb-bar/riscv-boom) (Berkeley's Out-of-Order processor generator). These generators are powerful in that they can generate an immensive variety of processors, in constrast to the typical fixed, one-configuration instances of Verilog-based hardware.
 - Often Verilog IP is hard to re-use because subsequent designs modify the original RTL in some subtle way that makes it slightly different. A generator can solve that by making those kinds of small changes parameters, allowing the 'library' code to be shared.
 - If you're interested, there will be an advanced Chisel tutorial that will go over using Chisel for generators.

On a side note: why is Chisel considered a domain-specific language (DSL)?
* A DSL could be considered as just an API with fancy symbols for names.
* There isn't anything in particular that prevents Chisel from being a pure API, though having operators (a key aspect of DSLs) makes expressing hardware more concise (e.g. x := y instead of x.connect(y)).


## Debugging Tips

Here are some tips to help you if you get stuck with something in Chisel.

### Wiki (Official User's Manual)

As of August 2017, the [Chisel3 Wiki](https://github.com/freechipsproject/chisel3/wiki) contains the official Chisel User's Manual. Feel free to consult it as an introduction to certain aspects of how to use Chisel or as a reference guide/cookbook.

### API

The [Chisel API documentation](http://chisel.eecs.berkeley.edu/api/) contains documentation on all the DSL keywords, APIs, functions, and classes of Chisel. It is a great reference to consult if you're not sure what a certain Chisel keyword is or how a certain function works. 

### Cheatsheets

The following cheatsheets may be useful for quickly referencing something or for remembering a particular idiom of piece of syntax.

* [Chisel3 Cheatsheet](http://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf) - overview of all the major functions and operators of Chisel. Best used in conjunction with the API reference above to see how each function/operator works.
* [ChiselSheet](https://github.com/freechipsproject/chisel3/wiki/ChiselSheet) - Scala vs Chisel vs Verilog and common idioms.

### Looking at generated Verilog/FIRRTL

If you are having trouble understanding the generated hardware and are comfortable with reading structural Verilog and/or FIRRTL (Chisel's IR which is comparable to a synthesis-only subset of Verilog), then you can try looking at the generated Verilog to see the result of executing the Chisel.

Here is an example below:

TODO: explain what's going on below (string interpreter)

In [ ]:
// A sample counter module for demonstration purposes.
class MyCounter extends Module {
  val io = IO(new Bundle {
        val in = Input(UInt(10.W))
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg := myReg + 1.U
  io.out := myReg

  // printing results
  printf("out=%d\n", io.out)
  printf(p"${io}\n")
}

We can view the resulting generated Verilog as shown below.

In [ ]:
// Viewing the Verilog for debugging
println(getVerilog(dut = () => new MyCounter))

TODO: maybe give quick overview of FIRRTL and using it as a debugging technique

In [ ]:
// Viewing the firrtl for debugging
println(chisel3.Driver.emit(() => new MyCounter))

### Error messages

Be careful of mixing up Scala-land (compile/elaboration-time) and Chisel-land (hardware/run-time) constructs inappropriately. Here are some common examples.

Run the following examples to experience the kinds of error messages that happen when you mix Scala-land and Chisel-land inappropriately. Then, try your hand at fixing the examples below so that they work correctly!

In [ ]:
// Using the = operator for hardware instead of the connection operator :=.
class BadModule extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(2.W))
  })
  val myReg = RegInit(UInt(2.W), 1.U)
  myReg = myReg + 1.U
  io.out = myReg
}

In [ ]:
// Using Scala-land constants instead of Chisel-land constants.
class BadCounter extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(8.W))
  })
  val myReg = RegInit(UInt(8.W), 0)
  when (myReg > 42) {
    myReg := 0
  } .otherwise {
    myReg := myReg + 1
  }
  io.out := myReg
}

In [ ]:
// Forgetting to wrap types in some sort of hardware: either a port like Input()
// or a hardware node like Wire()/Reg() or wrapping a module declaration in Module().
class BadAdder extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(8.W))
    val b = Input(UInt(8.W))
    val out = UInt(8.W)
  })
  io.out := io.a + io.b
}

class BadderAdder extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(8.W))
    val b = Input(UInt(8.W))
    val c = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })
  val adder1 = Module(new BadAdder)
  val adder2 = new BadAdder
  adder1.io.a := io.a
  adder1.io.b := io.b
  adder2.io.a := adder1.io.out
  adder2.io.b := io.c
  io.out := adder2.io.out
}

// See if we can get some Verilog.
println(getVerilog(dut = () => new BadderAdder))

In [ ]:
class BadCounterTwo extends Module {
  val io = IO(new Bundle {
    val out = Output(UInt(8.W))
  })
  val myReg = RegInit(UInt(8.W), 0.U)
  val regNext = UInt(8.W)
  when (myReg > 42.U) {
    regNext := 0.U
  } .otherwise {
    regNext := myReg + 2.U
  }
  myReg := regNext
  io.out := myReg
}

// See if we can get some Verilog.
println(getVerilog(dut = () => new BadCounterTwo))